In [6]:
import requests
import os
import re
import csv
from urllib.parse import urlparse
import pandas as pd

UA = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Python-requests downloader"

In [7]:
external_links_csv = pd.read_csv("external_links_8_from_red.csv")
external_links = external_links_csv['external_url']

external_links.to_csv("videos.csv", index=False, header=False)

In [3]:
def get_slug(url: str) -> str:
    """Extract slug from Redgifs URL."""
    path = urlparse(url).path.strip("/")
    return path.split("/")[-1]

def get_token() -> str:
    """Get temporary API token."""
    r = requests.get("https://api.redgifs.com/v2/auth/temporary",
                     headers={"User-Agent": UA, "Accept": "application/json"}, timeout=20)
    r.raise_for_status()
    return r.json()["token"]

def best_url(urls: dict) -> str | None:
    """Pick highest-quality available."""
    order = ["hd", "max", "sd", "mobile", "gif"]
    for key in order:
        if urls.get(key):
            return urls[key]
    return None

def fetch_gif_info(slug: str, token: str) -> dict:
    """Fetch clip metadata."""
    r = requests.get(f"https://api.redgifs.com/v2/gifs/{slug}",
                     headers={
                         "User-Agent": UA,
                         "Accept": "application/json",
                         "Authorization": f"Bearer {token}"
                     }, timeout=20)
    if r.status_code == 404:
        raise FileNotFoundError("Clip not found (private or removed).")
    r.raise_for_status()
    return r.json()["gif"]

def download(url: str, outpath: str):
    """Download video to file."""
    with requests.get(url, stream=True, headers={"User-Agent": UA}, timeout=60) as r:
        r.raise_for_status()
        with open(outpath, "wb") as f:
            for chunk in r.iter_content(chunk_size=1 << 14):
                if chunk:
                    f.write(chunk)

def save_redgifs(url_or_slug: str, token: str, outdir: str = ".") -> str:
    """Download single Redgifs video."""
    slug = get_slug(url_or_slug) if url_or_slug.startswith("http") else url_or_slug
    info = fetch_gif_info(slug, token)
    url = best_url(info.get("urls", {}))
    if not url:
        raise RuntimeError("No downloadable URL found.")
    title = info.get("title") or slug
    safe = re.sub(r"[^\w\-\.]+", "_", title).strip("_")
    fname = f"{safe or slug}.mp4"
    outpath = os.path.join(outdir, fname)
    print(f"Downloading {slug} → {outpath}\n{url}")
    download(url, outpath)
    return outpath

In [4]:

with open('videos.csv', newline="", encoding="utf-8") as f:
    reader = csv.reader(f)
    
    for row in reader:
        link = row[0].strip()
        print(link)

https://www.reddit.com/r/u_Environmental_Sail68/comments/11nfo8t/jackandjill_peachiikitten_lilykawaii_dirtydallas1/
https://www.reddit.com/r/SluttyConfessions/comments/tfucs4/how_i_f18_went_from_innocent_college_virgin_to/
https://www.reddit.com/r/SluttyConfessions/comments/tex5xg/how_i_f18_went_from_innocent_college_virgin_to/
https://www.reddit.com/r/SluttyConfessions/comments/tc0kxn/how_i_f18_went_from_innocent_college_virgin_to/
https://www.reddit.com/r/SluttyConfessions/comments/tbxv2a/how_i_f18_went_from_innocent_college_virgin_to/
https://www.reddit.com/r/SluttyConfessions/comments/tfmh7h/i_gave_a_much_younger_coworker_head_in_the/
https://www.reddit.com/r/SluttyConfessions/comments/nljzbf/f19_i_just_agreed_to_be_free_use_for_a_house_of_6/
https://www.reddit.com/r/SluttyConfessions/comments/n15k5n/my_exwife_blows_me_more_now_that_we_are_divorced/
https://www.reddit.com/r/SluttyConfessions/comments/t4at3m/when_i_was_in_high_school_i_f18_got_gangbanged_by/
https://www.reddit.com/r

In [5]:
def log_failed(link: str, fail_csv: str):
    """Append failed link to CSV."""
    exists = os.path.exists(fail_csv)
    with open(fail_csv, "a", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        if not exists:
            w.writerow(["failed_link"])
        w.writerow([link])
    print(f"Logged failed link → {fail_csv}")

if __name__ == "__main__":
    csv_file = "videos.csv"
    output_folder = "downloads"
    fail_csv = "external_links_2_from_red.csv"
    os.makedirs(output_folder, exist_ok=True)

    token = get_token()

    with open(csv_file, newline="", encoding="utf-8") as f:
        reader = csv.reader(f)
        for row in reader:
            if not row:
                continue
            link = row[0].strip()
            if not link or link.startswith("#"):
                continue
            try:
                save_redgifs(link, token, outdir=output_folder)
            except Exception as e:
                print(f"Failed {link}: {e}")
                log_failed(link, fail_csv)

Failed https://www.reddit.com/r/u_Environmental_Sail68/comments/11nfo8t/jackandjill_peachiikitten_lilykawaii_dirtydallas1/: 405 Client Error: Method Not Allowed for url: https://api.redgifs.com/v2/gifs/jackandjill_peachiikitten_lilykawaii_dirtydallas1
Logged failed link → external_links_2_from_red.csv
Failed https://www.reddit.com/r/SluttyConfessions/comments/tfucs4/how_i_f18_went_from_innocent_college_virgin_to/: 405 Client Error: Method Not Allowed for url: https://api.redgifs.com/v2/gifs/how_i_f18_went_from_innocent_college_virgin_to
Logged failed link → external_links_2_from_red.csv
Failed https://www.reddit.com/r/SluttyConfessions/comments/tex5xg/how_i_f18_went_from_innocent_college_virgin_to/: 405 Client Error: Method Not Allowed for url: https://api.redgifs.com/v2/gifs/how_i_f18_went_from_innocent_college_virgin_to
Logged failed link → external_links_2_from_red.csv
Failed https://www.reddit.com/r/SluttyConfessions/comments/tc0kxn/how_i_f18_went_from_innocent_college_virgin_to/: 